In [2]:
"""
Análisis financiero y de sensibilidad del LCOE y VAN para sistemas fotovoltaicos en distintos sitios.

Este script:
1. Carga la producción anual simulada por sitio desde un archivo CSV.
2. Calcula el LCOE (Levelized Cost of Energy) y VAN (Valor Actual Neto) base para cada sitio.
3. Evalúa el impacto de 5 parámetros críticos mediante un análisis de sensibilidad:
   - Tasa de descuento (FCR)
   - CapEx del sistema PV
   - Precio spot de la energía
   - Vida útil del inversor
   - Pérdidas del sistema
4. Genera gráficos tipo tornado por sitio para LCOE y VAN.
5. Exporta tablas resumen en formato CSV.

Requiere: pandas, numpy_financial, matplotlib
"""

import pandas as pd
import numpy_financial as npf
import matplotlib.pyplot as plt

# === Cargar archivo con producción anual por sitio ===
df = pd.read_csv("produccion_anual_por_sitio.csv")

# === Parámetros económicos base ===
capacidad_kw = 50000                              # Capacidad del sistema en kW
opex_anual_usd = 7 * capacidad_kw                 # Costos O&M anuales
vida_util = 20                                    # Vida útil del proyecto (años)
vida_inversor_base = 12                           # Vida útil del inversor
costo_inversor_usd_kw = 100                       # Costo de recambio del inversor por kW
precio_spot = 50                                  # Precio de venta de energía [USD/MWh]
capex_base = 758                                  # Costo de inversión inicial [USD/kW]
fcr_base = 0.08                                   # Fixed Charge Rate (tasa de descuento efectiva)
perdidas_base = 0.14                              # Pérdidas del sistema

# === Rango de valores para análisis de sensibilidad ===
parametros = {
    "FCR": [0.06, 0.10],
    "CapEx PV": [capex_base * 0.8, capex_base * 1.2],
    "Precio spot": [40, 60],
    "Vida inversor": [10, 20],
    "Pérdidas": [0.12, 0.16],
}

# === Colores personalizados para los gráficos ===
colores = {
    "FCR": "darkorange",
    "CapEx PV": "teal",
    "Precio spot": "cornflowerblue",
    "Vida inversor": "mediumseagreen",
    "Pérdidas": "lightcoral"
}

# === Función para calcular LCOE y VAN ===
def calcular_lcoe(prod_kwh, fcr, capex, spot, vida_inv, perdidas):
    prod_mwh = prod_kwh / 1000  # Conversión a MWh
    energia_util = prod_mwh * (1 - perdidas)
    ingresos_anuales = energia_util * spot
    flujos = [ingresos_anuales - opex_anual_usd] * vida_util

    if vida_inv < vida_util:
        flujos[vida_inv - 1] -= costo_inversor_usd_kw * capacidad_kw

    van = npf.npv(rate=fcr, values=flujos)
    lcoe = (fcr * capex * capacidad_kw + opex_anual_usd * vida_util) / (energia_util * vida_util)
    return lcoe, van  # LCOE en USD/MWh

# === Inicializar resultados ===
tabla_resultados = []
resumen_final = []

for _, row in df.iterrows():
    sitio = row["Sitio"]
    prod_anual_kwh = row["Produccion_Anual_kWh"]

    lcoe_base, van_base = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, precio_spot, vida_inversor_base, perdidas_base)

    resumen_final.append({
        "Sitio": sitio,
        "Producción Anual (MWh)": round(prod_anual_kwh / 1000, 2),
        "LCOE base (USD/MWh)": round(lcoe_base, 2),
        "VAN base (USD)": round(van_base, 2)
    })

    print(f"\n📍 {sitio.upper()} — Producción: {round(prod_anual_kwh / 1000):,} MWh")
    print(f"   🔹 LCOE base: {lcoe_base:.2f} USD/MWh")
    print(f"   💰 VAN base:  ${van_base:,.2f}")

    resultados_sens_lcoe = {}
    resultados_sens_van = {}

    for param, (low, high) in parametros.items():
        if param == "FCR":
            lcoe_low, van_low = calcular_lcoe(prod_anual_kwh, low, capex_base, precio_spot, vida_inversor_base, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual_kwh, high, capex_base, precio_spot, vida_inversor_base, perdidas_base)
        elif param == "CapEx PV":
            lcoe_low, van_low = calcular_lcoe(prod_anual_kwh, fcr_base, low, precio_spot, vida_inversor_base, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual_kwh, fcr_base, high, precio_spot, vida_inversor_base, perdidas_base)
        elif param == "Precio spot":
            lcoe_low, van_low = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, low, vida_inversor_base, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, high, vida_inversor_base, perdidas_base)
        elif param == "Vida inversor":
            lcoe_low, van_low = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, precio_spot, low, perdidas_base)
            lcoe_high, van_high = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, precio_spot, high, perdidas_base)
        elif param == "Pérdidas":
            lcoe_low, van_low = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, precio_spot, vida_inversor_base, low)
            lcoe_high, van_high = calcular_lcoe(prod_anual_kwh, fcr_base, capex_base, precio_spot, vida_inversor_base, high)

        resultados_sens_lcoe[param] = (lcoe_low, lcoe_high)
        resultados_sens_van[param] = (van_low, van_high)

        tabla_resultados.append({
            "Sitio": sitio,
            "Parámetro": param,
            "LCOE base": round(lcoe_base, 2),
            "LCOE low": round(lcoe_low, 2),
            "LCOE high": round(lcoe_high, 2),
            "VAN base (USD)": round(van_base, 2),
            "VAN low (USD)": round(van_low, 2),
            "VAN high (USD)": round(van_high, 2)
        })

    for metric, resultados, base, ylabel, filename_tag in [
        ("LCOE", resultados_sens_lcoe, lcoe_base, "LCOE (USD/MWh)", "lcoe"),
        ("VAN", resultados_sens_van, van_base, "VAN (USD)", "van")
    ]:
        fig, ax = plt.subplots(figsize=(10, 6))
        y_pos = range(len(parametros))
        valores = [v for pair in resultados.values() for v in pair]
        margen = (max(valores) - min(valores)) * 0.05
        ax.set_xlim(min(valores) - margen, max(valores) + margen)

        for i, param in enumerate(parametros):
            low, high = resultados[param]
            ancho = max(high, low) - min(high, low)
            ax.barh(i, ancho if ancho > 0.01 else 0.01, left=min(low, high),
                    color=colores[param], edgecolor='black')
            ax.text(min(low, high) - 0.01, i + 0.15, f"{low:,.2f}", ha='right', fontsize=9)
            ax.text(max(low, high) + 0.01, i - 0.15, f"{high:,.2f}", ha='left', fontsize=9)

        ax.axvline(base, color='red', linestyle='--', label=f"{metric} base: {base:,.2f}")
        ax.set_yticks(list(y_pos))
        ax.set_yticklabels(parametros)
        ax.set_xlabel(ylabel)
        ax.set_title(f"Análisis de sensibilidad {metric} - {sitio}")
        ax.grid(axis='x', linestyle='--', alpha=0.5)
        ax.legend()
        plt.tight_layout()
        plt.savefig(f"{sitio.lower()}_sensibilidad_{filename_tag}.png")
        plt.close()
        print(f"🖼️ Imagen guardada: {sitio.lower()}_sensibilidad_{filename_tag}.png")

# === Mostrar tabla resumen en consola ===
print("\n📋 Tabla resumen de resultados económicos:")
df_resumen = pd.DataFrame(resumen_final)
print(df_resumen.to_string(index=False))

# === Exportar solo la tabla completa de sensibilidad ===
pd.DataFrame(tabla_resultados).to_csv("tabla_sensibilidad_lcoe_y_van.csv", index=False)
print("\n📄 Archivos generados:")
print("✅ tabla_sensibilidad_lcoe_y_van.csv")



📍 CALAMA — Producción: 100,130 MWh
   🔹 LCOE base: 5.83 USD/MWh
   💰 VAN base:  $39,798,927.09
🖼️ Imagen guardada: calama_sensibilidad_lcoe.png
🖼️ Imagen guardada: calama_sensibilidad_van.png

📍 SALVADOR — Producción: 90,935 MWh
   🔹 LCOE base: 6.41 USD/MWh
   💰 VAN base:  $35,606,563.38
🖼️ Imagen guardada: salvador_sensibilidad_lcoe.png
🖼️ Imagen guardada: salvador_sensibilidad_van.png

📍 VALLENAR — Producción: 96,290 MWh
   🔹 LCOE base: 6.06 USD/MWh
   💰 VAN base:  $38,048,208.98
🖼️ Imagen guardada: vallenar_sensibilidad_lcoe.png
🖼️ Imagen guardada: vallenar_sensibilidad_van.png

📋 Tabla resumen de resultados económicos:
   Sitio  Producción Anual (MWh)  LCOE base (USD/MWh)  VAN base (USD)
  Calama               100129.67                 5.83     39798927.09
Salvador                90934.98                 6.41     35606563.38
Vallenar                96290.00                 6.06     38048208.98

📄 Archivos generados:
✅ tabla_sensibilidad_lcoe_y_van.csv
